In [ ]:
import warnings
warnings.filterwarnings('ignore')
import picaso.justdoit as jdi
import astropy.units as u

import numpy as np
import time
import matplotlib.pyplot as plt

In [ ]:
mhdeq='+0.0'
CtoOdeq='1.0'

filename_db="/Users/sagnickmukherjee/Documents/GitHub/picaso-dev/reference/opacities/ck_cx_cont_opacities.db"
ck_db='/Users/sagnickmukherjee/Documents/GitHub/Disequilibrium-picaso/reference/climate_INPUTS/sonora_2020_feh'+mhdeq+'_co_'+CtoOdeq+'.data.196'

opacity_ck = jdi.opannection(ck=True, ck_db=ck_db,filename_db=filename_db)


In [ ]:
Teff_ar = np.array([1000])
grav_ar = np.array([1000])
kzz=np.array([1e9])


gases_fly = ['CO','CH4','H2O','NH3','CO2','N2','HCN','H2','PH3','C2H2','Na','K','TiO','VO','FeH']

for iii in range(len(kzz)):
    T_eff= Teff_ar[0] 
    grav = grav_ar[0]
    nlevel = 91

    ## Brown Dwarf 
    T_star = "NA"
    semi_major = "NA" # AU
    self_consistent_kzz= False # Only needed if you want to do on-the-fly calculation with MLT Kzz
    cl_run = jdi.inputs(calculation="brown", climate = True)

    cl_run.gravity(gravity=grav, gravity_unit=u.Unit('m/(s**2)'))
    cl_run.T_eff(Teff=T_eff)




    
    temp_guess = np.zeros(shape=(nlevel))
    kz = np.zeros(shape =(nlevel))

    ## load a bobcat model
    pressure_dummy,temp_guess = np.loadtxt("/Users/sagnickmukherjee/Documents/software/picaso-dev/Evolution/structures_m+0.0/t"+str(Teff_ar[0])+"g"+str(grav_ar[0])+"nc_m0.0.dat",usecols=[1,2],unpack=True, skiprows = 1)
    pressure=pressure_dummy
    #pressure = np.logspace(np.log10(1.7e-3),np.log10(500),91)
    for i in range(len(temp_guess)):
        temp_guess[i] = temp_guess[i] # perturb the profile
        kz[i] = kzz[iii] #0

    
    nofczns = 1
    nstr = np.array([0,77,89,0,0,0]) # initial guess of convective zones
    rfacv = 0.0
    rfaci =1.0
    cloudy = False # not cloudy
    fsed = 0.0
        
    cl_run.inputs_climate(temp_guess= temp_guess, pressure= pressure, nstr = nstr, nofczns = nofczns , rfacv = rfacv, rfaci = rfaci, cloudy = cloudy, mh = mhdeq, CtoO = CtoOdeq,species = ['MgSiO3','Mg2SiO4','Fe'], fsed = fsed)
    pressure , temp, dtdp, nstr_new, flux_plus_final,quench_levels, df, all_profiles, all_kzz,opd_,g0_,w0_ = cl_run.run_climate_model(opacity_ck,save_all_profiles=True, save_all_kzz = True,diseq_chem = True, self_consistent_kzz =self_consistent_kzz, kz = kz,vulcan_run = False,on_fly=True,gases_fly=gases_fly,mhdeq=mhdeq,CtoOdeq=CtoOdeq)
    
    
    

In [ ]:
path = "/Users/sagnickmukherjee/Documents/GitHub/picaso/Benchmarking/onfly/"

filename= path+"tpfile/tp_allonfly_ph3deq_planet_"+str(Teff_ar[0])+"_grav_"+str(grav_ar[0])+"_mh_"+mhdeq+"_CtoO_"+CtoOdeq+"_kzz_1e9.dat"
np.savetxt(filename, np.transpose([pressure,temp]), header= str("nstr "+str(nstr_new)+" T_eff "+str(T_eff)+" grav "+ str(grav)+" kz "+str(kz[0])+" T_star "+str(T_star)+" semi-major[AU] "+str(semi_major)))        

csv = path+"profile/profile_allonfly_ph3deq_planet_"+str(Teff_ar[0])+"_grav_"+str(grav_ar[0])+"_mh_"+mhdeq+"_CtoO_"+CtoOdeq+"_kzz_1e9.dat"
df.to_csv(csv,sep='\t')

hist = path+"hist/hist_allonfly_ph3deq_planet_"+str(Teff_ar[0])+"_grav_"+str(grav_ar[0])+"_mh_"+mhdeq+"_CtoO_"+CtoOdeq+"_kzz_1e9.dat"
np.savetxt(hist, all_profiles)

hist = path+"hist/hist_kz_allonfly_ph3deq_planet_"+str(Teff_ar[0])+"_grav_"+str(grav_ar[0])+"_mh_"+mhdeq+"_CtoO_"+CtoOdeq+"_kzz_1e9.dat"
np.savetxt(hist, all_kzz)

In [ ]:
plt.figure(figsize=(10,10))
plt.ylabel("Pressure [Bars]",fontsize=20)
plt.xlabel("Temperature [K]",fontsize=20)
plt.ylim(300,1e-4)

path = "/Users/sagnickmukherjee/Documents/GitHub/picaso/Benchmarking/onfly/"
filename1= path+"tpfile/tp_allonfly_planet_"+str(800)+"_grav_"+str(316)+"_mh_+0.0_CtoO_1.0_kzz_1e9.dat"
pressure_dummy,temp_dummy = np.loadtxt("/Users/sagnickmukherjee/Documents/software/picaso-dev/Evolution/structures_m+0.0/t"+str(800)+"g"+str(316)+"nc_m0.0.dat",usecols=[1,2],unpack=True, skiprows = 1)
pressure_1,temp_1 = np.loadtxt(filename1,usecols=[0,1],unpack=True)
plt.semilogy(temp_dummy,pressure_dummy,linewidth=3,color="r",label="Eq, [M/H]=0.0")
plt.semilogy(temp_1,pressure_1,linestyle="--",color="r",linewidth=3,label=" DEq, [M/H]=0.0")


path = "/Users/sagnickmukherjee/Documents/GitHub/picaso/Benchmarking/onfly/"
filename1= path+"tpfile/tp_allonfly_planet_"+str(800)+"_grav_"+str(316)+"_mh_+1.0_CtoO_1.0_kzz_1e9.dat"
pressure_1,temp_1 = np.loadtxt(filename1,usecols=[0,1],unpack=True)
path = "/Users/sagnickmukherjee/Documents/GitHub/picaso/Benchmarking/onfly/"
filename1= path+"tpfile/tp_allonfly_planet_"+str(800)+"_grav_"+str(316)+"_mh_+1.0_CtoO_1.0_kzz_0.dat"
pressure_dummy,temp_dummy = np.loadtxt(filename1,usecols=[0,1],unpack=True)
plt.semilogy(temp_dummy,pressure_dummy,linewidth=3,color="g",label="Eq, [M/H]=+1.0")
plt.semilogy(temp_1,pressure_1,linestyle="--",color="g",linewidth=3,label="DEq, [M/H]=+1.0")


path = "/Users/sagnickmukherjee/Documents/GitHub/picaso/Benchmarking/onfly/"
filename1= path+"tpfile/tp_allonfly_planet_"+str(800)+"_grav_"+str(316)+"_mh_-1.0_CtoO_1.0_kzz_1e9.dat"
pressure_1,temp_1 = np.loadtxt(filename1,usecols=[0,1],unpack=True)
path = "/Users/sagnickmukherjee/Documents/GitHub/picaso/Benchmarking/onfly/"
filename1= path+"tpfile/tp_allonfly_planet_"+str(800)+"_grav_"+str(316)+"_mh_-1.0_CtoO_1.0_kzz_0.dat"
pressure_dummy,temp_dummy = np.loadtxt(filename1,usecols=[0,1],unpack=True)
plt.semilogy(temp_dummy,pressure_dummy,linewidth=3,color="b",label="Eq, [M/H]=-1.0")
plt.semilogy(temp_1,pressure_1,linestyle="--",color="b",linewidth=3,label="DEq, [M/H]=-1.0")





plt.minorticks_on()
plt.tick_params(axis='y',which='major',length =20, width=3,direction='in',labelsize=20)
plt.tick_params(axis='y',which='minor',length =10, width=2,direction='in',labelsize=20)
plt.tick_params(axis='x',which='major',length =20, width=3,direction='in',labelsize=20)
plt.tick_params(axis='x',which='minor',length =10, width=2,direction='in',labelsize=20)
plt.legend(fontsize=20)
plt.savefig("Benchmarking/onfly/onfly_metallicity_Kzz1e9.pdf")

In [ ]:
plt.figure(figsize=(10,10))
plt.ylabel("Pressure [Bars]",fontsize=20)
plt.xlabel("Temperature [K]",fontsize=20)
plt.ylim(300,1e-4)

path = "/Users/sagnickmukherjee/Documents/GitHub/picaso/Benchmarking/onfly/"
filename1= path+"tpfile/tp_allonfly_ph3eq_planet_"+str(1000)+"_grav_"+str(1000)+"_mh_+0.0_CtoO_1.0_kzz_1e9.dat"
pressure_1,temp_1 = np.loadtxt(filename1,usecols=[0,1],unpack=True)
plt.semilogy(temp_1,pressure_1,linestyle="--",color="r",linewidth=3,label=r"K$_{\rm zz}$=10$^9$ cm$^2$/s, Eq. PH$_3$, [M/H]=0.0")


path = "/Users/sagnickmukherjee/Documents/GitHub/picaso/Benchmarking/onfly/"
filename1= path+"tpfile/tp_allonfly_ph3deq_planet_"+str(1000)+"_grav_"+str(1000)+"_mh_+0.0_CtoO_1.0_kzz_1e9.dat"
pressure_1,temp_1 = np.loadtxt(filename1,usecols=[0,1],unpack=True)
plt.semilogy(temp_1,pressure_1,linestyle="--",color="g",linewidth=3,label=r"K$_{\rm zz}$=10$^9$ cm$^2$/s, DEq. PH$_3$, [M/H]=0.0")




plt.minorticks_on()
plt.tick_params(axis='y',which='major',length =20, width=3,direction='in',labelsize=20)
plt.tick_params(axis='y',which='minor',length =10, width=2,direction='in',labelsize=20)
plt.tick_params(axis='x',which='major',length =20, width=3,direction='in',labelsize=20)
plt.tick_params(axis='x',which='minor',length =10, width=2,direction='in',labelsize=20)
plt.legend(fontsize=20)
plt.savefig("Benchmarking/onfly/onfly_phosphine_Kzz1e9.pdf")

In [ ]:
plt.figure(figsize=(10,10))
plt.ylabel("Pressure [Bars]",fontsize=20)
plt.xlabel("Temperature [K]",fontsize=20)
plt.ylim(300,1e-4)

path = "/Users/sagnickmukherjee/Documents/GitHub/picaso/Benchmarking/onfly/"
filename1= path+"tpfile/tp_allonfly_planet_"+str(2200)+"_grav_"+str(1000)+"_eq.dat"
pressure_dummy,temp_dummy = np.loadtxt("/Users/sagnickmukherjee/Documents/software/picaso-dev/Evolution/structures_m+0.0/t"+str(2200)+"g"+str(1000)+"nc_m0.0.dat",usecols=[1,2],unpack=True, skiprows = 1)
pressure_1,temp_1 = np.loadtxt(filename1,usecols=[0,1],unpack=True)
plt.semilogy(temp_dummy,pressure_dummy,linewidth=3,color="k",label="Premixed Run")
plt.semilogy(temp_1,pressure_1,linestyle="--",color="r",linewidth=3,label=" On fly mix with CO,CH4,H2O,NH3,CO2,N2,HCN,H2,PH3,C2H2,Na,K,TiO,VO,FeH")


path = "/Users/sagnickmukherjee/Documents/GitHub/picaso/Benchmarking/onfly/"
filename1= path+"tpfile/tp_allonfly_planet_"+str(1000)+"_grav_"+str(1000)+"_eq.dat"
pressure_dummy,temp_dummy = np.loadtxt("/Users/sagnickmukherjee/Documents/software/picaso-dev/Evolution/structures_m+0.0/t"+str(1000)+"g"+str(1000)+"nc_m0.0.dat",usecols=[1,2],unpack=True, skiprows = 1)
pressure_1,temp_1 = np.loadtxt(filename1,usecols=[0,1],unpack=True)
plt.semilogy(temp_dummy,pressure_dummy,linewidth=3,color="k")
plt.semilogy(temp_1,pressure_1,linestyle="--",color="r",linewidth=3)


path = "/Users/sagnickmukherjee/Documents/GitHub/picaso/Benchmarking/onfly/"
filename1= path+"tpfile/tp_allonfly_planet_"+str(600)+"_grav_"+str(1000)+"_eq.dat"
pressure_dummy,temp_dummy = np.loadtxt("/Users/sagnickmukherjee/Documents/software/picaso-dev/Evolution/structures_m+0.0/t"+str(600)+"g"+str(1000)+"nc_m0.0.dat",usecols=[1,2],unpack=True, skiprows = 1)
pressure_1,temp_1 = np.loadtxt(filename1,usecols=[0,1],unpack=True)
plt.semilogy(temp_dummy,pressure_dummy,linewidth=3,color="k")
plt.semilogy(temp_1,pressure_1,linestyle="--",color="r",linewidth=3)


path = "/Users/sagnickmukherjee/Documents/GitHub/picaso/Benchmarking/onfly/"
filename1= path+"tpfile/tp_allonfly_planet_"+str(400)+"_grav_"+str(1000)+"_eq.dat"
pressure_dummy,temp_dummy = np.loadtxt("/Users/sagnickmukherjee/Documents/software/picaso-dev/Evolution/structures_m+0.0/t"+str(400)+"g"+str(1000)+"nc_m0.0.dat",usecols=[1,2],unpack=True, skiprows = 1)
pressure_1,temp_1 = np.loadtxt(filename1,usecols=[0,1],unpack=True)
plt.semilogy(temp_dummy,pressure_dummy,linewidth=3,color="k")
plt.semilogy(temp_1,pressure_1,linestyle="--",color="r",linewidth=3)

plt.annotate("400 K",(200,10),fontsize=20)
plt.annotate("600 K",(700,10),fontsize=20)
plt.annotate("1000 K",(500,1e-2),fontsize=20)
plt.annotate("2200 K",(1200,1e-2),fontsize=20)



plt.minorticks_on()
plt.tick_params(axis='y',which='major',length =20, width=3,direction='in',labelsize=20)
plt.tick_params(axis='y',which='minor',length =10, width=2,direction='in',labelsize=20)
plt.tick_params(axis='x',which='major',length =20, width=3,direction='in',labelsize=20)
plt.tick_params(axis='x',which='minor',length =10, width=2,direction='in',labelsize=20)
plt.legend(fontsize=10)
plt.savefig("Benchmarking/onfly/onfly_benchmarking.pdf")

In [ ]:
pressure_dummy,temp_guess = np.loadtxt("/Users/sagnickmukherjee/Documents/software/picaso-dev/Evolution/structures_m+0.0/t"+str(Teff_ar[0])+"g"+str(grav_ar[0])+"nc_m0.0.dat",usecols=[1,2],unpack=True, skiprows = 1)


plt.figure(figsize=(10,10))
plt.ylabel("Pressure [Bars]",fontsize=20)
plt.xlabel("Temperature [K]",fontsize=20)

plt.ylim(300,1e-4)
plt.semilogy(temp_guess,pressure_dummy,linewidth=2,color="k",label="Premixed Run")
plt.semilogy(temp,pressure,linestyle="--",linewidth=3,label=" On fly mix with CO,CH4,H2O,NH3,CO2,N2,HCN,H2,PH3,C2H2,Na,K")

plt.tick_params(axis='y',which='major',length =20, width=3,direction='in',labelsize=20)
plt.tick_params(axis='y',which='minor',length =10, width=2,direction='in',labelsize=20)
plt.tick_params(axis='x',which='major',length =20, width=3,direction='in',labelsize=20)
plt.tick_params(axis='x',which='minor',length =10, width=2,direction='in',labelsize=20)
plt.legend(fontsize=12)